In [1]:
import tensorflow as tf


In [3]:
tf.enable_eager_execution()

AttributeError: module 'tensorflow' has no attribute 'enable_eager_execution'

In [2]:
tf.__version__#current

'2.0.0-rc1'

In [4]:
tf.__version__#old

'1.5.0'

In [247]:
import os
import pandas as pd
import numpy as np

from tensorflow import keras
#from keras.models import Model
#from keras.layers import Input, Dense
#from keras import metrics
from tensorflow.keras.utils import to_categorical

In [248]:
data_path = os.path.expanduser('C:\\Users\\might\\Desktop\\jupyter notebooks\\u.data')
df = pd.read_csv(data_path, sep='\t', header=None)#used for keras NN_SVD

df2= pd.read_csv(data_path, sep='\t',names= 'user_id	movie_id	rating	timestamp'.split('	'))#, header=None)#used for DeepCTR


#df[2]= df[2].apply(lambda x: int(x>3))#since in df, no. vals holding 1 ratings:6110, 2 ratings:11370, 3 ratings:27145, 4 ratings:34174, 5 ratings: 21201 

shuffled_df = df.loc[np.random.randint(0, 100000, size=df.shape[0])]

#sm_train_df = shuffled_df[:80000]#small dataframe, trimming data points to random 2000 datapoints
#test_df= shuffled_df[-20000:]

#print('new trimmed dataset:', sm_train_df.shape, '\nnew test dataset:', test_df.shape)
print('shape of dataset:', shuffled_df.shape)

shape of dataset: (100000, 4)


In [14]:
##code from previous keras implementation
x_train_user_in = to_categorical(shuffled_df[0])#contains the one-hot encoded user_id data, shaped (batch_size, max(sm_df[0]))
x_train_movie_in = to_categorical(shuffled_df[1])#contains the one-hot encoded movie_id data, shaped (batch_size, max(sm_df[1]))

y_ratings= shuffled_df[2]
print('shape of x_train_user_in:', x_train_user_in.shape,'\nshape of x_train_movie_in:', x_train_movie_in.shape,
      '\nshape of x_train_user_in:',y_ratings.shape)

shape of x_train_user_in: (100000, 944) 
shape of x_train_movie_in: (100000, 1683) 
shape of x_train_user_in: (100000,)


In [33]:
x_train_user_in[0]#derived from df[0] --correspoding to first user in uids

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [40]:
np.where(x_train_user_in[0]==1)

(array([455], dtype=int64),)

In [250]:
##drawing code from deepCTR
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from deepctr.models import DeepFM
from deepctr.inputs import SparseFeat, DenseFeat, get_fixlen_feature_names, VarLenSparseFeat

df2= pd.read_csv(data_path, sep='\t',names= 'user_id	movie_id	rating	timestamp'.split('	'))#, header=None)#used for DeepCTR

sparse_features = ["movie_id", "user_id"]
y= ['rating']

In [251]:
df2.head(4)

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923


In [62]:
#list(np.sort(np.unique(df2['user_id'])))

In [252]:
for feat in sparse_features:
        lbe = LabelEncoder()
        df2[feat] = lbe.fit_transform(df2[feat])
    ##unique features for each sparse field
    
df2[sparse_features[1]].head(4)#derived from df2['user_id'] --correspoding to first user in uids

0    195
1    185
2     21
3    243
Name: user_id, dtype: int64

In [18]:
fixlen_feature_columns = [SparseFeat(feat, df2[feat].nunique()) for feat in sparse_features]

linear_feature_columns = fixlen_feature_columns

dnn_feature_columns = fixlen_feature_columns

In [151]:
print('Sparse feats & Dense feats:\n',linear_feature_columns,'\n', dnn_feature_columns)

Sparse feats & Dense feats:
 [SparseFeat(name='movie_id', dimension=1682, use_hash=False, dtype='int32', embedding_name='movie_id', embedding=True), SparseFeat(name='user_id', dimension=943, use_hash=False, dtype='int32', embedding_name='user_id', embedding=True)] 
 [SparseFeat(name='movie_id', dimension=1682, use_hash=False, dtype='int32', embedding_name='movie_id', embedding=True), SparseFeat(name='user_id', dimension=943, use_hash=False, dtype='int32', embedding_name='user_id', embedding=True)]


In [30]:
#from collections import namedtuple#testing inheritence with namedtuple class & super() function

In [31]:
#person = namedtuple('Person','name age gender')

In [58]:
#with_class = namedtuple('Person', 'name class gender age gender', rename=True)
#print(with_class._fields)

#two_ages = collections.namedtuple('Person', 'name age gender age', rename=True)
#print two_ages._fields


('name', '_1', 'gender', 'age', '_4')


In [141]:
#class MyParentClass():
 #   def __init__(self):
  #      self.var = 7
  #  def printout(self):
   #     print('output is :', self.var)

#class SubClass(MyParentClass):
 #   def __init__(self):
  #      super().__init__()

In [142]:
#a= SubClass()

In [144]:
#a.printout()

output is : 7


In [153]:
#features = build_input_features(linear_feature_columns + dnn_feature_columns)
feature_columns= linear_feature_columns + dnn_feature_columns
feature_columns

[SparseFeat(name='movie_id', dimension=1682, use_hash=False, dtype='int32', embedding_name='movie_id', embedding=True),
 SparseFeat(name='user_id', dimension=943, use_hash=False, dtype='int32', embedding_name='user_id', embedding=True),
 SparseFeat(name='movie_id', dimension=1682, use_hash=False, dtype='int32', embedding_name='movie_id', embedding=True),
 SparseFeat(name='user_id', dimension=943, use_hash=False, dtype='int32', embedding_name='user_id', embedding=True)]

* #### Following is line by line execution of deepctr.models.DeepFM

In [ ]:
#DeepFM(linear_feature_columns, dnn_feature_columns, embedding_size=8, use_fm=True, dnn_hidden_units=(128, 128),
#l2_reg_linear=0.00001, l2_reg_embedding=0.00001, l2_reg_dnn=0, init_std=0.0001, seed=1024,dnn_dropout=0, dnn_activation='relu',
#dnn_use_bn=False, task='binary')

In [169]:
input_features[feature_columns[0].name] = Input(shape=(1,), name='' + feature_columns[0].name, dtype=feature_columns[0].dtype)

In [174]:
input_features[feature_columns[0].name]#Inpute layer corresponding to 1st SparseFeat object

<tf.Tensor 'movie_id_1:0' shape=(None, 1) dtype=int32>

In [235]:
from collections import OrderedDict
from tensorflow.python.keras.layers import  Embedding, Input, Flatten
#from tensorflow.python.keras.regularizers import l2

#from .layers.sequence import SequencePoolingLayer


def build_input_features(feature_columns, include_varlen=True, mask_zero=True, prefix='',include_fixlen=True):
    input_features = OrderedDict()
    if include_fixlen:
        for fc in feature_columns:
            if isinstance(fc,SparseFeat):
                input_features[fc.name] = Input(
                    shape=(1,), name=prefix+fc.name, dtype=fc.dtype)
            elif isinstance(fc,DenseFeat):
                input_features[fc.name] = Input(
                    shape=(fc.dimension,), name=prefix + fc.name, dtype=fc.dtype)
    if include_varlen:
        for fc in feature_columns:
            if isinstance(fc,VarLenSparseFeat):
                input_features[fc.name] = Input(shape=(fc.maxlen,), name=prefix + 'seq_' + fc.name,
                                                      dtype=fc.dtype)
        if not mask_zero:
            for fc in feature_columns:
                input_features[fc.name+"_seq_length"] = Input(shape=(
                    1,), name=prefix + 'seq_length_' + fc.name)
                input_features[fc.name+"_seq_max_length"] = fc.maxlen


    return input_features

In [236]:
features= build_input_features(feature_columns)
features #Input feature layers obatined from SparseFeat parameters

OrderedDict([('movie_id',
              <tf.Tensor 'movie_id_11:0' shape=(None, 1) dtype=int32>),
             ('user_id',
              <tf.Tensor 'user_id_9:0' shape=(None, 1) dtype=int32>)])

In [189]:
inputs_list = list(features.values())
inputs_list

[<tf.Tensor 'movie_id_5:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'user_id_3:0' shape=(None, 1) dtype=int32>]

In [191]:
sparse_feature_columns = list(filter(lambda x:isinstance(x,SparseFeat), dnn_feature_columns)) if dnn_feature_columns else []

sparse_feature_columns

[SparseFeat(name='movie_id', dimension=1682, use_hash=False, dtype='int32', embedding_name='movie_id', embedding=True),
 SparseFeat(name='user_id', dimension=943, use_hash=False, dtype='int32', embedding_name='user_id', embedding=True)]

In [195]:
from tensorflow.python.keras.initializers import RandomNormal

def create_embedding_dict(sparse_feature_columns, varlen_sparse_feature_columns, embedding_size, init_std, seed, l2_reg,
                          prefix='sparse_', seq_mask_zero=True):
    if embedding_size == 'auto':
        print("Notice:Do not use auto embedding in models other than DCN")
        sparse_embedding = {feat.embedding_name: Embedding(feat.dimension, 6 * int(pow(feat.dimension, 0.25)),
                                                 embeddings_initializer=RandomNormal(
                                                     mean=0.0, stddev=init_std, seed=seed),
                                                 embeddings_regularizer=l2(l2_reg),
                                                 name=prefix + '_emb_' + feat.name) for feat in
                            sparse_feature_columns}
    else:

        sparse_embedding = {feat.embedding_name: Embedding(feat.dimension, embedding_size,
                                                 embeddings_initializer=RandomNormal(
                                                     mean=0.0, stddev=init_std, seed=seed),
                                                 embeddings_regularizer=l2(
                                                     l2_reg),
                                                 name=prefix + '_emb_'  + feat.name) for feat in
                            sparse_feature_columns}

    if varlen_sparse_feature_columns and len(varlen_sparse_feature_columns) > 0:
        for feat in varlen_sparse_feature_columns:
            # if feat.name not in sparse_embedding:
            if embedding_size == "auto":
                sparse_embedding[feat.embedding_name] = Embedding(feat.dimension, 6 * int(pow(feat.dimension, 0.25)),
                                                        embeddings_initializer=RandomNormal(
                                                            mean=0.0, stddev=init_std, seed=seed),
                                                        embeddings_regularizer=l2(
                                                            l2_reg),
                                                        name=prefix + '_seq_emb_' + feat.name,
                                                        mask_zero=seq_mask_zero)

            else:
                sparse_embedding[feat.embedding_name] = Embedding(feat.dimension, embedding_size,
                                                        embeddings_initializer=RandomNormal(
                                                            mean=0.0, stddev=init_std, seed=seed),
                                                        embeddings_regularizer=l2(
                                                            l2_reg),
                                                        name=prefix + '_seq_emb_' + feat.name,
                                                        mask_zero=seq_mask_zero)


    return sparse_embedding

def create_embedding_matrix(feature_columns,l2_reg,init_std,seed,embedding_size, prefix="",seq_mask_zero=True):
    sparse_feature_columns = list(
        filter(lambda x: isinstance(x, SparseFeat) and x.embedding, feature_columns)) if feature_columns else []
    varlen_sparse_feature_columns = list(
        filter(lambda x: isinstance(x, VarLenSparseFeat) and x.embedding, feature_columns)) if feature_columns else []
    sparse_emb_dict = create_embedding_dict(sparse_feature_columns, varlen_sparse_feature_columns, embedding_size, init_std, seed,
                                                 l2_reg, prefix=prefix + 'sparse',seq_mask_zero=seq_mask_zero)
    return sparse_emb_dict


In [199]:
#line 3 in input_from_feature_columns( ) function
embedding_dict = create_embedding_matrix(dnn_feature_columns, 0.00001, 0.0001, 1024, 8, prefix='',seq_mask_zero=True)
embedding_dict

{'movie_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x254edbea208>,
 'user_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x254eba407f0>}

In [206]:
#line 4 in input_from_feature_columns( ) function
def embedding_lookup(sparse_embedding_dict,sparse_input_dict,sparse_feature_columns,return_feat_list=(), mask_feat_list=()):
    embedding_vec_list = []
    for fc in sparse_feature_columns:
        feature_name = fc.name
        embedding_name = fc.embedding_name
        if len(return_feat_list) == 0  or feature_name in return_feat_list and fc.embedding:
            #if fc.use_hash:
            #    lookup_idx = Hash(fc.dimension,mask_zero=(feature_name in mask_feat_list))(sparse_input_dict[feature_name])
            lookup_idx = sparse_input_dict[feature_name]

            embedding_vec_list.append(sparse_embedding_dict[embedding_name](lookup_idx))

    return embedding_vec_list

In [207]:
sparse_feature_columns = list(filter(lambda x: isinstance(x, SparseFeat) and x.embedding, dnn_feature_columns)) if dnn_feature_columns else []

#features: OrderedDict([('movie_id',<tf.Tensor 'movie_id_7:0' shape=(None, 1) dtype=int32>), ...])
# output from build_input_features( ), line 1 in DeepFM() function

sparse_embedding_list = embedding_lookup(embedding_dict, features, sparse_feature_columns)
print('sparse_feature_columns:\n',sparse_feature_columns)
print('\nsparse_embedding_list:\n',sparse_embedding_list)

sparse_feature_columns:
 [SparseFeat(name='movie_id', dimension=1682, use_hash=False, dtype='int32', embedding_name='movie_id', embedding=True), SparseFeat(name='user_id', dimension=943, use_hash=False, dtype='int32', embedding_name='user_id', embedding=True)]

sparse_embedding_list:
 [<tf.Tensor 'sparse_emb_movie_id/Identity:0' shape=(None, 1, 8) dtype=float32>, <tf.Tensor 'sparse_emb_user_id/Identity:0' shape=(None, 1, 8) dtype=float32>]


In [210]:
#line 6 in input_from_Feature_Columns( )
def get_dense_input(features,feature_columns):
    dense_feature_columns = list(filter(lambda x:isinstance(x,DenseFeat),feature_columns)) if feature_columns else []
    dense_input_list = []
    for fc in dense_feature_columns:
        dense_input_list.append(features[fc.name])
    return dense_input_list

dense_value_list = get_dense_input(features,feature_columns)
dense_value_list

[]

In [216]:
features

OrderedDict([('movie_id',
              <tf.Tensor 'movie_id_7:0' shape=(None, 1) dtype=int32>),
             ('user_id',
              <tf.Tensor 'user_id_5:0' shape=(None, 1) dtype=int32>)])

In [218]:
#from deepctr.inputs 
def varlen_embedding_lookup(embedding_dict, sequence_input_dict, varlen_sparse_feature_columns):
    varlen_embedding_vec_dict = {}
    for fc in varlen_sparse_feature_columns:
        feature_name = fc.name
        embedding_name = fc.embedding_name
        if fc.use_hash:
            lookup_idx = Hash(fc.dimension, mask_zero=True)(sequence_input_dict[feature_name])
        else:
            lookup_idx = sequence_input_dict[feature_name]
        varlen_embedding_vec_dict[feature_name] = embedding_dict[embedding_name](lookup_idx)

    return varlen_embedding_vec_dict

def get_varlen_pooling_list(embedding_dict, features, varlen_sparse_feature_columns):
    pooling_vec_list = []
    for fc in varlen_sparse_feature_columns:
        feature_name = fc.name
        combiner = fc.combiner
        feature_length_name = feature_name + '_seq_length'
        if feature_length_name in features:
            vec = SequencePoolingLayer(combiner, supports_masking=False)(
            [embedding_dict[feature_name], features[feature_length_name]])
        else:
            vec = SequencePoolingLayer(combiner, supports_masking=True)(
            embedding_dict[feature_name])
        pooling_vec_list.append(vec)
    return pooling_vec_list

In [219]:
#line 2 in input_from_featre_columns( )
varlen_sparse_feature_columns = list(filter(lambda x: isinstance(x, VarLenSparseFeat), feature_columns)) if feature_columns else []
print('varlen_sparse_feature_columns: ', varlen_sparse_feature_columns)

sequence_embed_dict = varlen_embedding_lookup(embedding_dict,features,varlen_sparse_feature_columns)
print('\nsequence_embed_dict: ',sequence_embed_dict)

sequence_embed_list = get_varlen_pooling_list(sequence_embed_dict, features, varlen_sparse_feature_columns)
print('\nsequence_embed_list: ', sequence_embed_list)

varlen_sparse_feature_columns:  []

sequence_embed_dict:  {}

sequence_embed_list:  []


In [221]:
#line 3 in DeepFM( ) function
#features: output from build_input_features( ) from line 1 in DeepFM()
#feature_columns: dnn_feature_columns
def input_from_feature_columns(features,feature_columns, embedding_size, l2_reg, init_std, seed,prefix='',seq_mask_zero=True,support_dense=True):


    sparse_feature_columns = list(filter(lambda x:isinstance(x,SparseFeat),feature_columns)) if feature_columns else []
    varlen_sparse_feature_columns = list(filter(lambda x: isinstance(x, VarLenSparseFeat), feature_columns)) if feature_columns else []

    embedding_dict = create_embedding_matrix(feature_columns,l2_reg,init_std,seed,embedding_size, prefix=prefix,seq_mask_zero=seq_mask_zero)
    
    sparse_embedding_list = embedding_lookup(
        embedding_dict, features, sparse_feature_columns)
    dense_value_list = get_dense_input(features,feature_columns)
    if not support_dense and len(dense_value_list) >0:
        raise ValueError("DenseFeat is not supported in dnn_feature_columns")

    sequence_embed_dict = varlen_embedding_lookup(embedding_dict,features,varlen_sparse_feature_columns)
    sequence_embed_list = get_varlen_pooling_list(sequence_embed_dict, features, varlen_sparse_feature_columns)
    sparse_embedding_list += sequence_embed_list

    return sparse_embedding_list, dense_value_list

In [223]:
sparse_embedding_list, dense_value_list = input_from_feature_columns(features,dnn_feature_columns,
     embedding_size=8, l2_reg=0.00001,init_std=0.0001,seed=1024)
print('sparse_embedding_list: ', sparse_embedding_list)
print('\ndense_value_list: ', dense_value_list)

sparse_embedding_list:  [<tf.Tensor 'sparse_emb_movie_id_1/Identity:0' shape=(None, 1, 8) dtype=float32>, <tf.Tensor 'sparse_emb_user_id_1/Identity:0' shape=(None, 1, 8) dtype=float32>]

dense_value_list:  []


**Now define a SVD_NN() along the lines with DeepFM() function, using/customising Some of the above methods/functionalities**

In [116]:
from collections import OrderedDict
from tensorflow.python.keras.layers import  Embedding, Input, Dense
import os
import tensorflow as tf
import pandas as pd
import numpy as np
#from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
#from deepctr.models import DeepFM

from deepctr.inputs import SparseFeat#DenseFeat, get_fixlen_feature_names, VarLenSparseFeat




#insert this to from .layers.sequence import SequencePoolingLayer
def build_input_features(feature_columns, include_varlen=True, prefix='',include_fixlen=True):
    input_features = OrderedDict()
    if include_fixlen:
        for fc in feature_columns:
            if isinstance(fc,SparseFeat):
                input_features[fc.name] = Input(
                    shape=(fc.dimension,), name=prefix+fc.name, dtype=fc.dtype)# Here shape changed from shape=(1,) to shape= (fc.dimesnion,)
            #elif isinstance(fc,DenseFeat):
             #   input_features[fc.name] = Input(
              #      shape=(fc.dimension,), name=prefix + fc.name, dtype=fc.dtype)
                
    return input_features


def NN_svd(fixlen_feature_columns, num_factors=100):
    features = build_input_features(fixlen_feature_columns)
    
    inputs= list(features.values())
    
    hid_layer_1= Dense(num_factors)(inputs[0])
    hid_layer_2= Dense(num_factors)(inputs[1])
    
    merge_layer = tf.keras.layers.dot([hid_layer_1, hid_layer_2], axes=1)
    
    predictions = Dense(5, activation='softmax')(merge_layer)
    
    model = tf.keras.models.Model(inputs=inputs, outputs=predictions)
    
    return model
    
    

In [273]:
feature= build_input_features(fixlen_feature_columns)
features

OrderedDict([('movie_id',
              <tf.Tensor 'movie_id_11:0' shape=(None, 1) dtype=int32>),
             ('user_id',
              <tf.Tensor 'user_id_9:0' shape=(None, 1) dtype=int32>)])

In [12]:
data_path = os.path.expanduser('C:\\Users\\might\\Desktop\\jupyter notebooks\\u.data')
df2= pd.read_csv(data_path, sep='\t',names= 'user_id	movie_id	rating	timestamp'.split('	'))#, header=None)#used for DeepCTR

sparse_features = ["movie_id", "user_id"]
y= ['rating']

In [14]:
#This counts unique values & encodes existing value to new lable in progression
for feat in sparse_features:
        lbe = LabelEncoder()
        df2[feat] = lbe.fit_transform(df2[feat])
    ##unique features for each sparse field
    
fixlen_feature_columns = [SparseFeat(feat, df2[feat].nunique()) for feat in sparse_features]
print(fixlen_feature_columns)

[SparseFeat(name='movie_id', dimension=1682, use_hash=False, dtype='int32', embedding_name='movie_id', embedding=True), SparseFeat(name='user_id', dimension=943, use_hash=False, dtype='int32', embedding_name='user_id', embedding=True)]


In [150]:
from keras.utils import to_categorical

train, test = train_test_split(df2, test_size=0.2)

train_model_input = [to_categorical(train[fc.name].values, num_classes= fc.dimension) for fc in fixlen_feature_columns]#includes values from only data[user_id], data[movie_id]
test_model_input = [to_categorical(test[fc.name].values, num_classes= fc.dimension) for fc in fixlen_feature_columns]#includes values from only data[user_id], data[movie_id]


#abc= to_categorical(train_model_input[0], num_classes= fixlen_feature_columns[0].dimension)

In [110]:
model = NN_svd(fixlen_feature_columns)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
movie_id (InputLayer)           [(None, 1682)]       0                                            
__________________________________________________________________________________________________
user_id (InputLayer)            [(None, 943)]        0                                            
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 100)          168300      movie_id[0][0]                   
__________________________________________________________________________________________________
dense_13 (Dense)                (None, 100)          94400       user_id[0][0]                    
____________________________________________________________________________________________

In [71]:
abc= model.get_weights()

In [74]:
abc[0].shape

(1682, 100)

In [73]:
len(abc)

6

In [111]:
model.compile(loss= 'categorical_crossentropy',optimizer="sgd", metrics=['mae', 'accuracy'])

In [104]:
#categorical labels for softmax
y_cat = to_categorical(train[y])

y_cat= y_cat[:,1:]#stripping 0th column to shape (None,5)

In [113]:
model.fit(train_model_input, y_cat, batch_size=64, epochs=8, verbose=2, validation_split=0.2,)

Train on 64000 samples, validate on 16000 samples
Epoch 1/8
64000/64000 - 2s - loss: 1.4678 - mae: 0.2994 - accuracy: 0.3417 - val_loss: 1.4694 - val_mae: 0.2995 - val_accuracy: 0.3406
Epoch 2/8
64000/64000 - 2s - loss: 1.4675 - mae: 0.2993 - accuracy: 0.3417 - val_loss: 1.4693 - val_mae: 0.2994 - val_accuracy: 0.3406
Epoch 3/8
64000/64000 - 2s - loss: 1.4674 - mae: 0.2992 - accuracy: 0.3417 - val_loss: 1.4691 - val_mae: 0.2993 - val_accuracy: 0.3406
Epoch 4/8
64000/64000 - 2s - loss: 1.4672 - mae: 0.2991 - accuracy: 0.3417 - val_loss: 1.4690 - val_mae: 0.2993 - val_accuracy: 0.3406
Epoch 5/8
64000/64000 - 2s - loss: 1.4670 - mae: 0.2991 - accuracy: 0.3417 - val_loss: 1.4688 - val_mae: 0.2993 - val_accuracy: 0.3406
Epoch 6/8
64000/64000 - 2s - loss: 1.4668 - mae: 0.2991 - accuracy: 0.3417 - val_loss: 1.4686 - val_mae: 0.2992 - val_accuracy: 0.3406
Epoch 7/8
64000/64000 - 2s - loss: 1.4665 - mae: 0.2991 - accuracy: 0.3417 - val_loss: 1.4683 - val_mae: 0.2992 - val_accuracy: 0.3406
Epoch

**SVD as a module**
    * Following svd.SVD is just the method to output model, whereas the data preparation has tp be done usual beforehand.

In [146]:
import os
os.getcwd()

'C:\\Users\\might\\Desktop\\jupyter notebooks'

In [71]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
#from deepctr.models import DeepFM


c:\users\might\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [9]:
data_path = os.path.expanduser('C:\\Users\\might\\Desktop\\jupyter notebooks\\u.data')
df2= pd.read_csv(data_path, sep='\t',names= 'user_id	movie_id	rating	timestamp'.split('	'))#, header=None)#used for DeepCTR

sparse_features = ["movie_id", "user_id"]
y= ['rating']

In [67]:
from deepctr.inputs import SparseFeat##Choosing SpareFeat Class for fixlen_feature_Column creates different input shape (None,1)
fixlen_feature_columns = [SparseFeat(feat, df2[feat].nunique()) for feat in sparse_features]
print(fixlen_feature_columns)

[SparseFeat(name='movie_id', dimension=1682, use_hash=False, dtype='int32', embedding_name='movie_id', embedding=True), SparseFeat(name='user_id', dimension=943, use_hash=False, dtype='int32', embedding_name='user_id', embedding=True)]


In [68]:
from deepctr.models import svd

model = svd.SVD(fixlen_feature_columns, task='multiclass')
model.summary()
#Here the Input layer is shaped (None, 1) with fixlen_feature_columns created using SparseFeat class
#(due to structure of DeepCTR.Inputs.build_feature_input)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
movie_id (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 100)          200         movie_id[0][0]                   
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 100)          200         user_id[0][0]                    
____________________________________________________________________________________________

**Changed the fixlen_feature_columns definition using DenseFeat instead of SparseFeat**

In [78]:
from deepctr.inputs import DenseFeat
fixlen_feature_columns = [DenseFeat(feat, df2[feat].nunique()) for feat in sparse_features]
print(fixlen_feature_columns)

[DenseFeat(name='movie_id', dimension=1682, dtype='float32'), DenseFeat(name='user_id', dimension=943, dtype='float32')]


In [79]:
from deepctr.models import svd

model = svd.SVD(fixlen_feature_columns, task='multiclass')
model.summary()
#Now the Input layer is shaped (None, 1682) with fixlen_feature_columns created using DenseFeat class

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
movie_id (InputLayer)           [(None, 1682)]       0                                            
__________________________________________________________________________________________________
user_id (InputLayer)            [(None, 943)]        0                                            
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 100)          168300      movie_id[0][0]                   
__________________________________________________________________________________________________
dense_13 (Dense)                (None, 100)          94400       user_id[0][0]                    
____________________________________________________________________________________________

In [80]:
model.compile(loss= 'categorical_crossentropy',optimizer="sgd", metrics=['mae', 'accuracy'])

In [81]:
from tensorflow.keras.utils import to_categorical

train, test = train_test_split(df2, test_size=0.2)

train_model_input = [to_categorical(train[fc.name].values, num_classes= fc.dimension) for fc in fixlen_feature_columns]#includes values from only data[user_id], data[movie_id]
test_model_input = [to_categorical(test[fc.name].values, num_classes= fc.dimension) for fc in fixlen_feature_columns]#includes values from only data[user_id], data[movie_id]

#categorical labels for softmax
y_cat = to_categorical(train[y])

y_cat= y_cat[:,1:]#stripping 0th column to shape (None,5)

In [82]:
model.fit(train_model_input, y_cat, batch_size=64, epochs=8, verbose=2, validation_split=0.2,)

Train on 64000 samples, validate on 16000 samples
Epoch 1/8
64000/64000 - 3s - loss: 1.5100 - mae: 0.3086 - accuracy: 0.3396 - val_loss: 1.4689 - val_mae: 0.3023 - val_accuracy: 0.3479
Epoch 2/8
64000/64000 - 2s - loss: 1.4723 - mae: 0.3013 - accuracy: 0.3412 - val_loss: 1.4607 - val_mae: 0.2997 - val_accuracy: 0.3479
Epoch 3/8
64000/64000 - 2s - loss: 1.4695 - mae: 0.2999 - accuracy: 0.3412 - val_loss: 1.4592 - val_mae: 0.2991 - val_accuracy: 0.3479
Epoch 4/8
64000/64000 - 2s - loss: 1.4688 - mae: 0.2996 - accuracy: 0.3412 - val_loss: 1.4586 - val_mae: 0.2988 - val_accuracy: 0.3479
Epoch 5/8
64000/64000 - 2s - loss: 1.4685 - mae: 0.2994 - accuracy: 0.3412 - val_loss: 1.4583 - val_mae: 0.2987 - val_accuracy: 0.3479
Epoch 6/8
64000/64000 - 2s - loss: 1.4682 - mae: 0.2993 - accuracy: 0.3412 - val_loss: 1.4580 - val_mae: 0.2986 - val_accuracy: 0.3479
Epoch 7/8
64000/64000 - 2s - loss: 1.4679 - mae: 0.2993 - accuracy: 0.3412 - val_loss: 1.4577 - val_mae: 0.2986 - val_accuracy: 0.3479
Epoch

**Importing svd_complete.py**

* Following svd_complete.SVD_C is a class containing two methods
      1. To prepare dataset & abstract the initial few steps done as rotuine to feed model.fit
      2. To output model

In [84]:
from deepctr.models import svd_complete

In [90]:
print('Data_path: ',data_path, '\nsparse_features: ',sparse_features)

Data_path:  C:\Users\might\Desktop\jupyter notebooks\u.data 
sparse_features:  ['movie_id', 'user_id']


In [93]:
model, (tr_x,tr_y), (t_x,t_y) = svd_complete.SVD_C.prepare_data(data_path, sparse_features=sparse_features, task='binary')

MemoryError: 